# Robot Programming with Lisp - 5. More Functional Programming
**Lexical Scope, Closures, Recursion, Macros**

## Overview

* Lexical Scopes
* Closures
* Recursion
* Macros
* Organizational

## The let environment

In [ ]:
;; let
(let ((a 1)
      (b 2))
     (values a b))

In [ ]:
(values a b)

In [ ]:
(defvar some-var 'global)
(let ((some-var 'outer))
     (let ((some-var 'inter))
          (format t "some-var inner: ~a~%" some-var))
     (format t "some-var outer: ~a~%" some-var))
(format t "global-var: ~a~%" some-var)

In [ ]:
;; let*
(let ((a 4)
      (a^2 (expt a 2)))
     (values a a^2))

In [ ]:
(let* ((a 4)
       (a^2 (expt a 2)))
      (values a a^2))

## Lexical Variables

In Lisp, non-global variable values are, when possible, **determined at
compile time**. They are **bound lexically**, i.e. they are bound to the
code they’re defined in, not to the run-time state of the program. 

This is one single let block, therefore lexical-var is the same everywhere in the block.

In [ ]:
;; Riddle 1
(let* ((lexical-var 304)
       (some-lambda (lambda () (+ lexical-var 100))))
      (setf lexical-var 4)
      (funcall some-lambda))

In [ ]:
;; Lexical scope with lambda and defun
(defun return-x (x)
    (let ((x 304))
         x))

(return-x 3)

lambda-s and defun-s create lexical local variables per default.

In [ ]:
;; More Examples
(let* ((lexical-var 304)
       (some-lambda (lambda () (+ lexical-var 100))))
      (setf lexical-var 4)
      (funcall some-lambda))

In [ ]:
lexical-var

In [ ]:
(let ((another-var 304)
      (another-lambda (lambda () (+ another-var 100))))
     (setf another-var 4)
     (funcall another-lambda))

In [ ]:
(let ((other-lambda (lambda () (+ other-var 100))))
     (setf other-var 4)
     (funcall other-lambda))

In [ ]:
other-var

In [ ]:
(describe 'other-var)

In [ ]:
(let ((some-var 304))
     (defun some-fun () (+ some-var 100))
     (setf some-var 4)
     (funcall #'some-fun))

;; defun is compiled into
;; (defmacro-mundanely defun (&environment env name args &body body)
;;     (multiple-value-bind (forms decls doc) (parse-body body)
;;         (let* ((lambda-guts `(,args ...))
;;                (lambda `(lambda ,@lambda-guts)) ...

In [ ]:
;; Riddle 2
(let ((lex 'initial-value))
     
  (defun return-lex ()
    lex)
     
  (defun return-lex-arg (lex)
    (return-lex))
     
  (format t "return-lex: ~a~%"
          (return-lex))
     
  (format t "return-lex-arg: ~a~%"
          (return-lex-arg 'new-value))
     
  (format t "return-lex again: ~a~%"
          (return-lex)))

In [ ]:
;; Riddle 3
(defvar dyn 'initial-value)

(defun return-dyn ()
    dyn)

(defun return-dyn-arg (dyn)
    (return-dyn))

In [ ]:
(format t "return-dyn: ~a~%" (return-dyn))

(format t "return-dyn-arg: ~a~%" (return-dyn-arg 'new-value))

(format t "return-dyn again: ~a~%" (return-dyn))

## Local Function Definitions

In [ ]:
;; flet

(defun some-pseudo-code ()
    (flet ((do-something (arg-1)
               (format t "doing something ~a now...~%" arg-1)))
          (format t "hello.~%")
          (do-something "nice")
          (format t "hello once again.~%")
          (do-something "evil")))

In [ ]:
(some-pseudo-code)

In [ ]:
(do-something)

In [ ]:
;; flet, labels

(let* ((lexical-var 304)
       (some-lambda (lambda () (+ lexical-var 100))))
      (let ((lexical-var 4))
           (funcall some-lambda)))

In [ ]:
(let ((lexical-var 304))
     (flet ((some-function () (+ lexical-var 100)))
           (let ((lexical-var 4))
                (some-function))))

In [ ]:
(labels ((first-fun () (format t "inside FIRST~%"))
         (second-fun ()
                     (format t "inside SECOND~%")
                     (first-fun)))
        (second-fun))

## Closures

In [ ]:
;; Counter

(defun increment-counter ()
  (let ((counter 0))
    (incf counter)))
(increment-counter)
(increment-counter)

In [ ]:
(defvar *counter* 0)
(defun increment-counter-function ()
    (incf *counter*))
(increment-counter-function)
(increment-counter-function)

In [ ]:
(setf *counter* 5)

In [ ]:
(increment-counter-function)

Closure is a function that, in addition to its specific functionality, also
encloses its lexical environment. **Encapsulation!**

In [ ]:
;; Counter As Closure

(let ((counter 0))
     (defun increment-counter-closure ()
         (incf counter)))
(increment-counter-closure)
(increment-counter-closure)

In [ ]:
#'increment-counter-function

In [ ]:
#'increment-counter-closure

In [ ]:
counter

In [ ]:
;; Creating Closures

(let ((input (read)))
     (lambda () (print input)))

In [ ]:
(funcall *)

In [ ]:
(alexandria:curry #'expt 10)

In [ ]:
(funcall * 3)

In [ ]:
(defvar *input* (read))

In [ ]:
(lambda () (print *input*))

## Recursion

In [ ]:
;; Primitive Example

(defun dummy-recursion (my-list)
    (when my-list
        (dummy-recursion (rest my-list))))

In [ ]:
(trace dummy-recursion)

In [ ]:
(dummy-recursion '(1 2 3 4 5))

In [ ]:
;; Primitive Example 2

(defun print-list (list)
    (format t "list: ~a" list)
    (when list
        (format t " -> first: ~a~%" (first list))
        (print-list (rest list))))

In [ ]:
(print-list '(1 2 3))

In [ ]:
;; reminder
(mapl (lambda (list)
              (format t "list: ~a -> first: ~a~%" list (first list)))
      '(1 2 3))

In [ ]:
;; Length of a List: calculate on the way up

(defun my-length (a-list)
    (if (null a-list)
        0
        (+ 1 (my-length (rest a-list)))))

In [ ]:
(trace my-length)

In [ ]:
(my-length '(5 a 3 8))

In [ ]:
;; Length of a list: calculate on the way down — Accumulators

(defun my-length-inner (a-list accumulator)
    (if (null a-list)
        accumulator
        (my-length-inner (rest a-list) (1+ accumulator))))

In [ ]:
(trace my-length-inner)

In [ ]:
(my-length-inner '(5 a 3 8) 0)

In [ ]:
;; Length of a list: passing initial accumulator value

(defun my-length-outer (a-list)
    (my-length-inner a-list 0))

In [ ]:
(my-length-outer '(5 a 3 8))

In [ ]:
;; use &optional for initial and default values
(defun my-length-acc (a-list &optional (accumulator 0))
    (if (null a-list)
        accumulator
        (my-length-acc (rest a-list) (1+ accumulator))))

In [ ]:
(trace my-length-acc)

In [ ]:
(my-length-acc '(6 3 nj ws))

In [ ]:
;; Tail Recursion Optimization

(trace my-length-acc my-length)

In [ ]:
(my-length '(a b c))

In [ ]:
(my-length-acc '(a b c))

In [ ]:
(proclaim '(optimize speed))

In [ ]:
(defun my-length (a-list)
    (if (null a-list)
        0
        (+ 1 (my-length (rest a-list)))))

(defun my-length-acc (a-list &optional (accumulator 0))
    (if (null a-list)
        accumulator
        (my-length-acc (rest a-list) (1+ accumulator))))

In [ ]:
(my-length-acc '(a b c))

In [ ]:
(my-length '(a b c))

In [ ]:
;; What Does This Function Do?

(defun sigma (n)
    (labels ((sig (c n)
                  (declare (type fixnum n c))
                  (if (zerop n)
                      c
                      (sig (the fixnum (+ n c))
                           (the fixnum (- n 1))))))
            (sig 0 n)))

In [ ]:
(trace sigma)

In [ ]:
(sigma 5)

In [ ]:
(defun expects-float (arg-1)
    (declare (type single-float arg-1))
    (/ arg-1 5))

In [ ]:
(expects-float 3.0)

In [ ]:
(defun expects-fixnum (arg-1)
    (declare (type fixnum arg-1))
    (the ratio (/ arg-1 100)))

In [ ]:
(expects-fixnum 5)

## Macros - Generating Code

In [ ]:
'(if t 'yes 'no)

In [ ]:
(eval *)

In [ ]:
`(if t 'yes 'no)

In [ ]:
`((+ 1 2) ,(+ 3 4) (+ 5 6))

In [ ]:
(let ((x 26))
     `(if ,(oddp x)
          'yes
          'no))

In [ ]:
;; Double Quotes

''(+ 1 5)

In [ ]:
(eval *)

In [ ]:
(eval *)

In [ ]:
'`(a ,(+ 1 2))

In [ ]:
(eval *)

In [ ]:
`'(a ,(+ 1 2))

## Defining Macros

### defmacro

In [ ]:
(defun x^3-fun (x)
    (format t "type of X is ~a~%" (type-of x))
    (* x x x))

In [ ]:
(x^3-fun 4)

In [ ]:
(defmacro x^3-macro (x)
    (format t "type of X is ~a~%" (type-of x))
    (* x x x))

In [ ]:
(x^3-macro 4)

In [ ]:
(x^3-macro (+ 2 2))

In [ ]:
(defun use-x^3 (a)
    (x^3-macro a))

### macroexpand

In [ ]:
(defmacro x^3-backquote (x)
    (format t "type of X is ~a~%" (type-of x))
    `(* ,x ,x ,x))

In [ ]:
(defun use-x^3 (a)
    (x^3-backquote a))

In [ ]:
(use-x^3 4)

In [ ]:
(macroexpand '(x^3-backquote 4))

In [ ]:
(x^3-backquote (+ 2 2))

In [ ]:
(macroexpand '(x^3-backquote (+ 2 2)))

### defmacro continued

Macros transform code into other code by means of code.

In [ ]:
(defmacro x^3-let (x)
    (format t "type of X is ~a~%" (type-of x))
    `(let ((z ,x))
          (* z z z)))

In [ ]:
(x^3-let (+ 2 2))

In [ ]:
(macroexpand '(x^3-let (+ 2 2)))

### Macro arguments

In [ ]:
(defmacro test-macro (&whole whole arg-1
                             &optional (arg-2 1) arg-3)
    (format t "whole: ~a~%" whole)
    (format t "arg-1: ~a~%" arg-1)
    (format t "arg-2: ~a~%arg-3: ~a~%" arg-2 arg-3)
    `',whole)

In [ ]:
(macroexpand '(test-macro something))

In [ ]:
(test-macro something)

In [ ]:
(eval *)

### Built-in Macros

```lisp
(defmacro-mundanely when (test &body forms)
    `(if ,test (progn ,@forms) nil))

(defmacro-mundanely prog1 (result &body body)
    (let ((n-result (gensym)))
         `(let ((,n-result ,result))
               ,@body
               ,n-result)))

(defmacro-mundanely ignore-errors (&rest forms)
    `(handler-case (progn ,@forms)
         (error (condition) (values nil condition))))
```

### More Applications

```lisp
(defmacro get-time ()
    `(the unsigned-byte (get-internal-run-time)))

(defmacro definline (name arglist &body body)
    `(progn (declaim (inline ,name))
         (defun ,name ,arglist ,@body)))
```

In [ ]:
(defmacro info (message &rest args)
    (when (eq *release-or-debug* :debug)
        `(format *standard-output* ,message ,@args)))

In [ ]:
(defparameter *release-or-debug* :DEBUG)
(info "bla")

### A Better Example

In [ ]:
(defmacro square (&whole form arg)
    (if (atom arg)
        `(expt ,arg 2)
        (case (car arg)
            (square (if (= (length arg) 2)
                        `(expt ,(nth 1 arg) 4)
                        form))
            (expt (if (= (length arg) 3)
                      (if (numberp (nth 2 arg))
                          `(expt ,(nth 1 arg) ,(* 2 (nth 2 arg)))
                          `(expt ,(nth 1 arg) (* 2 ,(nth 2 arg))))
                      form))
            (otherwise `(expt ,arg 2)))))

In [ ]:
(macroexpand '(square (square 3)))

In [ ]:
(macroexpand '(square (expt 123 4)))

## Links

Functional programmer Bible (available for free):

http://www.paulgraham.com/onlisp.html

## Organizational

Assignment Points: 10 

Due Date: 30.11., Wednesday, 23:59 AM CEST

Next Class: 01.12., 14:15